<a href="https://colab.research.google.com/github/jovanadobreva/Labs-I2DS/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [1]:
# !gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

### Import the required libraries

In [2]:
import pandas as pd

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [3]:
data = pd.read_csv('ElectricCarData.csv')

In [4]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


### Encode string variables

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            103 non-null    object 
 1   Model            103 non-null    object 
 2   AccelSec         103 non-null    float64
 3   TopSpeed_KmH     103 non-null    int64  
 4   Range_Km         103 non-null    int64  
 5   Efficiency_WhKm  103 non-null    int64  
 6   FastCharge_KmH   103 non-null    object 
 7   RapidCharge      103 non-null    object 
 8   PowerTrain       103 non-null    object 
 9   PlugType         103 non-null    object 
 10  BodyStyle        103 non-null    object 
 11  Segment          103 non-null    object 
 12  Seats            103 non-null    int64  
 13  PriceEuro        103 non-null    int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 11.4+ KB


In [6]:
from sklearn.preprocessing import LabelEncoder

def label_encode_dataframe(df):
    label_encoders = {}

    for col in data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    return data, label_encoders

In [7]:
encoded_data, encoders = label_encode_dataframe(data)

In [8]:
encoded_data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,30,46,4.6,233,450,161,50,1,0,2,7,3,5,55480
1,31,33,10.0,160,270,167,7,1,2,2,1,2,5,30000
2,23,0,4.7,210,400,181,36,1,0,2,2,3,5,56440
3,2,101,6.8,180,360,206,32,1,2,2,6,3,5,68040
4,9,78,9.5,145,170,168,2,1,2,2,1,1,4,32997


## Split the dataset for training and testing in ratio 80:20

In [9]:
from sklearn.model_selection import train_test_split

target = 'PriceEuro'
X = encoded_data.drop([target], axis=1)
Y = encoded_data[target]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [10]:
from sklearn.tree import DecisionTreeRegressor

dec_tree_reg_model = DecisionTreeRegressor(max_depth=6, min_samples_split=5, max_features="sqrt")
dec_tree_reg_model.fit(X_train, Y_train)

DecisionTreeRegressor(max_depth=6, max_features='sqrt', min_samples_split=5)

## Predict the outcomes for X test

In [11]:
y_pred_dec_tree_reg = dec_tree_reg_model.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse_dtr = mean_squared_error(Y_test, y_pred_dec_tree_reg)
mae_dtr = mean_absolute_error(Y_test, y_pred_dec_tree_reg)
r2_dtr = r2_score(Y_test, y_pred_dec_tree_reg)

print(f"Mean Squared Error (MSE): {mse_dtr}")
print(f"Mean Absolute Error (MAE): {mae_dtr}")
print(f"R² Score: {r2_dtr}")

Mean Squared Error (MSE): 220867187.65736857
Mean Absolute Error (MAE): 9826.98044217687
R² Score: 0.7214921781435913


## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [13]:
from xgboost import XGBRegressor

xgbr_model = XGBRegressor(n_estimators=200, max_depth=7, learning_rate=0.01, objective="reg:squarederror")
xgbr_model.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Predict the outcomes for X test

In [14]:
y_pred_xgbr = xgbr_model.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [15]:
mse_xgbr = mean_squared_error(Y_test, y_pred_xgbr)
mae_xgbr = mean_absolute_error(Y_test, y_pred_xgbr)
r2_xgbr = r2_score(Y_test, y_pred_xgbr)

print(f"Mean Squared Error (MSE): {mse_xgbr}")
print(f"Mean Absolute Error (MAE): {mae_xgbr}")
print(f"R² Score: {r2_xgbr}")

Mean Squared Error (MSE): 72645197.66044907
Mean Absolute Error (MAE): 6183.5366443452385
R² Score: 0.9083962440490723


## Compare the performances of both model for at least three regression metircs

In [19]:
performance_comparison = pd.DataFrame({
    "Metric": ["MSE", "MAE", "R2"],
    "DecisionTreeRegressor": [mse_dtr, mae_dtr, r2_dtr],
    "XGBRegressor": [mse_xgbr, mae_xgbr, r2_xgbr]
})
performance_comparison["DecisionTreeRegressor"] = performance_comparison["DecisionTreeRegressor"].map('{:.2f}'.format)
performance_comparison["XGBRegressor"] = performance_comparison["XGBRegressor"].map('{:.2f}'.format)
print(performance_comparison)

  Metric DecisionTreeRegressor XGBRegressor
0    MSE          220867187.66  72645197.66
1    MAE               9826.98      6183.54
2     R2                  0.72         0.91


In [ ]:
# Better